# ERA-PageRank

## Dependencies

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

import paths

from TRMM import TRMM
from ERA import ERA

from ModelHelpers import ModelHelpers
from ModelERAv2 import ModelERAv2
from ModelERAv3 import ModelERAv3

# force autoreload of external modules on save
%load_ext autoreload
%autoreload 2
%matplotlib inline

Using TensorFlow backend.


## Constants and Configuration

In [2]:
YEARS = range(1979, 2018)
YEARS_TRAIN = range(1979, 2011)
YEARS_DEV = range(2011, 2013)
YEARS_TEST = range(2013, 2018)

EPOCHS = 100
PATIENCE = 0

# this can later be changed to any earlier date
PREDICT_ON = '{}-05-22'

In [29]:
# setup the hyperparameters for the model to be built

TUNING = {
        'aggregation_resolution': 1.0,
        'config_build': {
            'batch_norm': True,
            'conv_activation': 'relu',
            'conv_dropout': 0.0,
            'conv_filters': [30, 30, 30],
            'conv_kernels': [3, 3, 3],
            'conv_pooling': [0, 0, 0, 0],
            'conv_strides': [1, 1, 1],
            'conv_kernel_regularizer': (None, 0.02),
            'conv_recurrent_regularizer': (None, 0.02),
            'dense_dropout': 0.0,
            'dense_nodes': [1024, 1024, 1024],
            'dense_activation': 'relu',
            'dense_kernel_regularizer': (None, 0.02),
            'learning_rate': 0.1,
            'loss': 'mean_squared_error',
            'lstm_filters': [30, 30, 30],
            'lstm_kernels': [3, 3, 3],
            'lstm_strides': [1, 1, 1],
            'lstm_activation': 'relu',
            'lstm_recurrent_activation': 'hard_sigmoid',
            'lstm_recurrent_dropout': 0.0,
            'lstm_recurrent_dropout': 0.0,
            'lstm_recurrent_activation': 'hard_sigmoid',
            'optimizer': 'rmsprop',
            'padding': 'same'
        },
        'config_fit': {
            'batch_size': 1,
            'epochs': EPOCHS,
            'lr_plateau': (0.5, 10, 0.0001),
            'patience': PATIENCE,
            'tensorboard': True,
            'validation_split': 0.1
        },
        'objective_onsets': True,
        'predict_on': PREDICT_ON,
        'prediction_sequence': 20,
        'prediction_offset': 0,
        'years': YEARS,
        'years_train': YEARS_TRAIN,
        'years_dev': YEARS_DEV,
        'years_test': YEARS_TEST,
    }

## Building the ERA model

### Loading the ERA dataset

In [30]:
# prepare onset dates and prediction timestamps
onset_dates, onset_ts = ModelHelpers.load_onset_dates(version='v2', objective=True if TUNING['objective_onsets'] else False)

prediction_ts = ModelHelpers.generate_prediction_ts(TUNING['predict_on'], TUNING['years'], onset_dates=onset_dates, sequence_length=TUNING['prediction_sequence'], sequence_offset=TUNING['prediction_offset'])

# setup a filter function
# this later prevents any data after the prediction timestamp from being fed as input
def filter_fun(df, year):
    return ModelHelpers.filter_until(df, onset_ts[year])

prediction_ts[1979]

[(298512000.0, 293328000.0),
 (298425600.0, 293241600.0),
 (298339200.0, 293155200.0),
 (298252800.0, 293068800.0),
 (298166400.0, 292982400.0),
 (298080000.0, 292896000.0),
 (297993600.0, 292809600.0),
 (297907200.0, 292723200.0),
 (297820800.0, 292636800.0),
 (297734400.0, 292550400.0),
 (297648000.0, 292464000.0),
 (297561600.0, 292377600.0),
 (297475200.0, 292291200.0),
 (297388800.0, 292204800.0),
 (297302400.0, 292118400.0),
 (297216000.0, 292032000.0),
 (297129600.0, 291945600.0),
 (297043200.0, 291859200.0),
 (296956800.0, 291772800.0),
 (296870400.0, 291686400.0),
 (296784000.0, 291600000.0)]

In [31]:
# load the ERA dataset for the pre-monsoon period (MAM)

era_temp, era_hum = ERA.load_dataset(
    TUNING['years'],
    invalidate=False,
    version='v4',
    filter_fun=filter_fun,
    aggregation_resolution=TUNING['aggregation_resolution'])

era_temp[1979].unstack()

> Loading from cache...


time        289094400                                                  \
longitude       62.25       63.25       64.25       65.25       66.25   
latitude                                                                
5.0        298.216884  298.318262  298.638974  298.911529  299.046742   
6.0        297.680349  297.683097  297.954516  298.410419  298.673364   
7.0        297.535570  297.584478  297.594763  297.833684  298.221383   
8.0        297.777409  297.851263  297.646512  297.550911  297.843612   
9.0        298.020258  298.162722  297.873943  297.697028  297.851153   
10.0       298.085177  298.344518  298.154769  297.977443  298.060017   
11.0       297.764750  298.092598  298.048390  297.916012  297.820986   
12.0       297.152550  297.332613  297.223162  297.007853  296.842758   
13.0       296.453381  296.478398  296.192361  295.725488  295.500279   
14.0       296.009194  295.884714  295.636249  295.348869  295.341519   
15.0       295.324417  295.154046  295.296159  295.624457  296.117876   
16.0       294.682182  294.918306  295.570294  296.458789  297.236435   
17.0       294.430601  295.175026  295.995097  296.926375  297.578719   
18.0       293.943939  294.681304  295.638690  296.769672  297.491619   
19.0       294.014804  294.397215  295.409504  296.622677  297.236600   
20.0       294.179262  294.207301  295.056689  296.307105  297.018576   
21.0       294.045782  293.602072  294.299941  295.709527  296.757331   
22.0       294.094247  293.196246  293.483160  295.037437  296.254724   
23.0       294.413927  293.221531  293.311757  294.421211  295.544103   
24.0       294.301268  293.359914  293.338523  294.056412  294.951681   
25.0       294.412545  294.135175  294.339322  294.907665  295.950642   
26.0       295.600943  295.502226  296.005327  296.834668  297.622434   
27.0       297.495135  297.011199  297.030067  296.857869  296.936659   
28.0       299.128532  298.284239  297.925775  296.968801  295.175519   
29.0       298.023527  297.042051  296.712328  296.990466  294.575693   
30.0       294.017349  295.793501  295.328877  294.788561  293.554299   
31.0       292.233139  294.245805  294.429658  293.198632  292.474792   
32.0       290.682695  292.862984  292.363449  291.497003  291.626062   
33.0       287.647846  290.067204  289.427914  289.527190  289.653535   
34.0       282.540063  286.428672  288.164088  288.290569  287.832033   
35.0       276.585370  279.820791  283.438563  285.968244  286.745616   
36.0       274.531107  275.243714  276.482090  278.276688  280.403998   
37.0       274.184463  274.490804  275.002242  275.807258  277.455217   
38.0       273.932444  274.457155  275.127901  276.144660  278.517364   
39.0       272.239634  272.855625  273.604256  274.758823  277.461991   

time                                                                   \
longitude       67.25       68.25       69.25       70.25       71.25   
latitude                                                                
5.0        299.216839  299.487331  299.647929  299.667542  299.704642   
6.0        298.841366  299.000071  299.319977  299.606342  299.604368   
7.0        298.326226  298.505691  298.928932  299.399699  299.577272   
8.0        297.904603  298.144238  298.568164  299.168594  299.427014   
9.0        297.869199  298.052860  298.429808  298.901809  299.297297   
10.0       298.021952  298.125150  298.268224  298.646488  299.145996   
11.0       297.751438  297.902382  298.101319  298.378579  298.722344   
12.0       296.740438  297.050827  297.593254  298.063473  298.381925   
13.0       295.674671  296.193650  297.074769  297.776970  298.401204   
14.0       295.660273  296.310067  297.146237  297.736492  298.223494   
15.0       296.640037  297.126875  297.505798  297.786761  298.133652   
16.0       297.618896  297.827376  297.956244  298.033526  298.074717   
17.0       297.986081  298.170154  298.244145  298.210413  298.072907   
18.0       297.810236  298.107462  298.111164  298.087552

In [7]:
for year in YEARS:
    print(era_temp[year].shape)

(1225, 110)
(1225, 98)
(1225, 96)
(1225, 102)
(1225, 109)
(1225, 99)
(1225, 96)
(1225, 106)
(1225, 101)
(1225, 101)
(1225, 97)
(1225, 88)
(1225, 101)
(1225, 109)
(1225, 102)
(1225, 96)
(1225, 103)
(1225, 106)
(1225, 111)
(1225, 103)
(1225, 85)
(1225, 88)
(1225, 90)
(1225, 91)
(1225, 107)
(1225, 84)
(1225, 109)
(1225, 92)
(1225, 107)
(1225, 92)
(1225, 84)
(1225, 92)
(1225, 90)
(1225, 97)
(1225, 93)
(1225, 98)
(1225, 97)
(1225, 100)
(1225, 91)


### Train-Test Split

In [32]:
X_train, y_train, X_test, y_test, X_dev, y_dev = ModelERAv3.train_test_split(
    # an arbitrary number of features could be fed in here
    # pagerank will be added a a further feature later on
    [era_temp, era_hum],
    prediction_ts,
    onset_ts,
    years=TUNING['years'],
    years_train=TUNING['years_train'],
    years_test=TUNING['years_test'],
    years_dev=TUNING['years_dev'])

Processed 1979 (21, 61, 35, 35, 2)
Processed 1980 (21, 61, 35, 35, 2)
Processed 1981 (21, 61, 35, 35, 2)
Processed 1982 (21, 61, 35, 35, 2)
Processed 1983 (21, 61, 35, 35, 2)
Processed 1984 (21, 61, 35, 35, 2)
Processed 1985 (21, 61, 35, 35, 2)
Processed 1986 (21, 61, 35, 35, 2)
Processed 1987 (21, 61, 35, 35, 2)
Processed 1988 (21, 61, 35, 35, 2)
Processed 1989 (21, 61, 35, 35, 2)
Processed 1990 (21, 61, 35, 35, 2)
Processed 1991 (21, 61, 35, 35, 2)
Processed 1992 (21, 61, 35, 35, 2)
Processed 1993 (21, 61, 35, 35, 2)
Processed 1994 (21, 61, 35, 35, 2)
Processed 1995 (21, 61, 35, 35, 2)
Processed 1996 (21, 61, 35, 35, 2)
Processed 1997 (21, 61, 35, 35, 2)
Processed 1998 (21, 61, 35, 35, 2)
Processed 1999 (21, 61, 35, 35, 2)
Processed 2000 (21, 61, 35, 35, 2)
Processed 2001 (21, 61, 35, 35, 2)
Processed 2002 (21, 61, 35, 35, 2)
Processed 2003 (21, 61, 35, 35, 2)
Processed 2004 (21, 61, 35, 35, 2)
Processed 2005 (21, 61, 35, 35, 2)
Processed 2006 (21, 61, 35, 35, 2)
Processed 2007 (21, 

In [13]:
X_train[:, :, :, :, 0].mean()

-2.0873196094643057e-15

In [14]:
y_train

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
       13, 14, 15, 16, 17, 18, 19, 20,  0,  1,  2,  3,  4,  5,  6,  7,  8,
        9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,  0,  1,  2,  3,  4,
        5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,  0,
        1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
       14, 15, 16, 17, 18, 19, 20,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
       10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,  0,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,  0,  1,
        2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
       15, 16, 17, 18, 19, 20,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
       11, 12, 13, 14, 15

### Build a model

In [22]:
# build a model based on the above tuning

model, config, history = ModelHelpers.run_config(
    ModelERAv3,
    TUNING,
    X_train,
    y_train,
    invalidate=True,
    evaluate=None,
    validation_data=(X_dev, y_dev) if TUNING['years_dev'] else None,
    cache_id=0,
    version='E4')

>> Instantiating model
>> Building model
> input_shape: (61, 35, 35, 2)
>> Fitting model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 61, 35, 35, 2)     0         
_________________________________________________________________
pre-conv-0 (TimeDistributed) (None, 61, 35, 35, 30)    570       
_________________________________________________________________
pre-conv-0-dropout (TimeDist (None, 61, 35, 35, 30)    0         
_________________________________________________________________
pre-conv-norm-0 (BatchNormal (None, 61, 35, 35, 30)    120       
_________________________________________________________________
pre-conv-1 (TimeDistributed) (None, 61, 35, 35, 30)    8130      
_________________________________________________________________
pre-conv-1-dropout (TimeDist (None, 61, 35, 35, 30)    0         
_____________________________________________________

KeyboardInterrupt: 

In [ ]:
# evaluate the latest state of the model above

print('Train:', model.evaluate(X_train, y_train), model.predict(X_train))
print('Dev:', model.evaluate(X_dev, y_dev), model.predict(X_dev))
print('Test:', model.evaluate(X_test, y_test), model.predict(X_test))

In [ ]:
# evaluate the model with the lowest validation error over all epochs

best_model = model.load_model(which='best')

print('Train:', best_model.evaluate(X_train, y_train), best_model.predict(X_train))
print('Dev:', best_model.evaluate(X_dev, y_dev), best_model.predict(X_dev))
print('Test:', best_model.evaluate(X_test, y_test), best_model.predict(X_test))

## Extending the ERA model with PageRank

### Loading the TRMM dataset

In [ ]:
# TRMM is only used for calculating the PageRank parameters

trmm_data = TRMM.load_dataset(
    range(1998, 2018),
    range(3, 6),
    aggregation_resolution=TUNING['aggregation_resolution'],
    timestamp=True,
    invalidate=False,
    version='v2',
    filter_fun=filter_fun)

### Calculating PageRank

In [ ]:
# calculate extreme events

extreme_events = TRMM.extract_extreme_events(trmm_data, quantile=0.9)

extreme_events.head(10)

In [ ]:
# calculate the sync and count matrices from extreme events

sync_matrix, count_matrix, runtime = TRMM.calculate_sync_matrix(extreme_events, 'MAM')

In [ ]:
# create a climate network graph and calculate centrality measures
graph = TRMM.generate_graph(count_matrix, quantile=0.95)
_, _, pagerank = TRMM.calculate_centrality(graph)

pagerank.pivot(index='lat', columns='lon', values='val')

### Building a new model

In [ ]:
era_temp[2017].head(10)

In [ ]:
# we need to get the pagerank matrix into the same shape as the above data (a "time-series")
# the easiest ways for this will be to copy the data until we have the same shape
# there would probably be more efficient ways though
# TODO: it would also be possible to use different pagerank data for each year
# TODO: maybe based only on the years before?

pagerank_transformed = pagerank[['lat', 'lon', 'val']]
pagerank_transformed = pagerank_transformed.set_index(['lat', 'lon'])

# create a "time-series" of grids
for i in range(0, 82):
    pagerank_transformed[i] = pagerank_transformed['val']

pagerank_transformed.head()

In [ ]:
# copy the "time-series" once for each year
pr_input = {}
for year in YEARS:
    pr_input[year] = pagerank_transformed

pr_input[1979].shape

In [ ]:
# rebuild train, dev and test sets
# now containing the additional PageRank feature

X_train_pr, y_train_pr, X_test_pr, y_test_pr, X_dev_pr, y_dev_pr, unstacked = ModelERAv2.train_test_split(
    [era_temp, era_hum, pr_input],
    prediction_ts,
    onset_ts,
    years=TUNING['years'],
    years_train=TUNING['years_train'],
    years_test=TUNING['years_test'],
    years_dev=TUNING['years_dev'])

In [ ]:
# build a model based on the above tuning

model_pr, config_pr, history_pr = ModelHelpers.run_config(
    ModelERAv2,
    TUNING,
    X_train_pr,
    y_train_pr,
    invalidate=True,
    evaluate=None,
    validation_data=(X_dev_pr, y_dev_pr) if TUNING['years_dev'] else None,
    cache_id=1,
    version='E4')

In [ ]:
# evaluate the latest state of the model above

print('Train:', model_pr.evaluate(X_train_pr, y_train_pr), model_pr.predict(X_train_pr))
print('Dev:', model_pr.evaluate(X_dev_pr, y_dev_pr), model_pr.predict(X_dev_pr))
print('Test:', model_pr.evaluate(X_test_pr, y_test_pr), model_pr.predict(X_test_pr))

In [ ]:
# evaluate the model with the lowest validation error over all epochs

best_model_pr = model_pr.load_model(which='best')

print('Train:', best_model_pr.evaluate(X_train_pr, y_train_pr), best_model_pr.predict(X_train_pr))
print('Dev:', best_model_pr.evaluate(X_dev_pr, y_dev_pr), best_model_pr.predict(X_dev_pr))
print('Test:', best_model_pr.evaluate(X_test_pr, y_test_pr), best_model_pr.predict(X_test_pr))